Dependencies

In [2]:
!pip install graphviz
!pip install pandas


Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping


In [3]:
import graphviz
import pandas as pd
import re
from rolestereotype import RoleStereotype as rs
from collections import Counter
from collections import defaultdict

The Pattern class (loads .pattern DP-CORE format)

In [4]:
def calls(a,b):
    return a,b,{'arrowhead':'vee', 'style':'dashed', 'label':'calls'}

def creates(a,b):
    return a,b,{'dir':"back", 'arrowtail':"diamond", 'label':'creates'}

def uses(a,b):
    return a,b,{'arrowhead':"vee", 'style':"dashed", 'label':'uses'}

def has(a,b):
    return a,b,{'dir':"back", 'arrowtail':"odiamond", 'label':'has'}

def references(a,b):
    return a,b,{'arrowhead':"vee", 'label':'references'}

def inherits(a,b):
    return b,a,{'dir':"back", 'arrowtail':"empty"}

CONNECTIONS = {
    'calls': calls,
    'creates': creates,
    'uses': uses,
    'has': has,
    'references': references,
    'inherits': inherits,
}

class Connection:
    def __init__(self, name, participants):
        self.name = name
        self.participants = participants
    @property
    def edge(self):
        return CONNECTIONS[self.name](self.participants[0],self.participants[1])

class Member:
    def __init__(self, name, classname, annot=None, stereotype=None):
        self.name = name
        self.classname = classname
        self.annot = annot
        self.stereotype = stereotype
    @property
    def label(self):
        annot = f"«{self.annot}» " if self.annot else ''
        return f"<{{{annot}{self.classname}|}}>"
    @property
    def attrs(self):
        attr = {'shape':'record'}
        if self.stereotype:
            attr['fillcolor']=self.stereotype.bcolor
            attr['color']=self.stereotype.fcolor
            attr['style']='filled'
        return attr
    def __str__(self):
        return ', '.join(str(s) for s in [self.annot, self.classname, self.stereotype] if s)

class Pattern:
    abstractionMapping = {
        'Normal': None,
        'Abstract': 'abstract',
        'Interface': 'interface',
        'Abstracted': 'abstracted',
        'Any': None
    }
    def __init__(self, lines):
        idx = 0
        self.name = lines[idx].strip()
        self.members = list()
        self.connections = list()
        idx += 1
        line = lines[idx]
        while line.strip() != 'End_Members':
            line = line.split()
            self.members.append(Member(name=line[0], annot=Pattern.abstractionMapping[line[1]], classname=' '.join(line[2:])))
            idx += 1
            line = lines[idx]
        idx += 1
        line = lines[idx]
        while line.strip() != 'End_Connections':
            line = line.split()
            self.connections.append(Connection(line[1], (line[0], line[2])))
            idx += 1
            line = lines[idx]
    def graph(self):
        dot = graphviz.Digraph(name=self.name)
        dot.attr(fontname='Arial')
        for m in self.members:
            dot.node(m.name, m.label, _attributes=m.attrs)
        for c in self.connections:
            a,b,attr = c.edge
            dot.edge(a, b, _attributes=attr)

        return dot



Load pattern descriptions

Note: current best representation of patterns:

- bridge: bridge-v1
- adapter: object-adapter
- factory method: factory-method-v3
- abstract factory: (unknown)

In [5]:
patternNames = [
    'abstract-factory-v1', 
    'abstract-factory-v2',
    'bridge-v1',
    'bridge-v2', 
    'builder', 
    'class-adapter',
    'command', 
    'composite',
    'decorator',
    'factory-method-v1',
    'factory-method-v2',
    'factory-method-v3',
    'object-adapter',
    'observer', 
    'proxy',
    'visitor'
]

patterns = dict()
for name in patternNames:
    file = open(f'pattern-descriptions/{name}.pattern', 'r')
    lines = file.readlines()
    pattern = Pattern(lines)
    file.close()
    patterns[name] = pattern
    # graph = pattern.graph()
    # print(graph.source)
    # graph.render(outfile=f'pattern-descriptions/graphs/{name}.png')

Load ground truth data, containing class types (e.g., abstract? interface?) and role stereotypes

In [6]:
gt = pd.read_csv('role-stereotypes.csv')

gt_k9 = gt[gt['case_name'] == 'K-9 Mail']

gt_k9[gt_k9['classname'] == 'AuthType'].iloc[0]

def getGroundTruth(classname, dframe):
    return dframe[dframe['classname'] == classname]

def isInGroundTruth(classname, dframe):
    return getGroundTruth(classname, dframe).shape[0] > 0

def roleStereotype(classname, dframe):
    return rs.from_str(getGroundTruth(classname, dframe).iloc[0]['label']) if isInGroundTruth(classname, dframe) else None

def abstraction(classname, dframe):
    gt = getGroundTruth(classname, dframe)
    if gt.shape[0] == 0:
        return None
    if gt.iloc[0]['isInterface']:
        return 'interface'
    elif gt.iloc[0]['isAbstract']:
        return 'abstract'
    else:
        return 'concrete'

print(isInGroundTruth('AuthType', gt_k9))
print(isInGroundTruth('Hello', gt_k9))

print(roleStereotype('AuthType', gt_k9), abstraction('AuthType', gt_k9))

gt_k9[gt_k9['classname'].str.contains('Builder')]

True
False
Information Holder concrete


,case_name,case_url,version_tag,full_classname,classname,isStaticClass,isInterface,isInnerClass,isClass,isEnum,classPublicity,isAbstract,label
111,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.mail.store.imap.UidSearchCommandBu...,UidSearchCommandBuilder,False,False,False,True,False,default,False,Information Holder
146,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.mail.internet.MimeMessage$MimeMess...,MimeMessage$MimeMessageBuilder,False,False,True,True,False,private,False,Structurer
304,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.search.SqlQueryBuilder,SqlQueryBuilder,False,False,False,True,False,public,False,Service Provider
434,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.IdentityHeaderBuilder,IdentityHeaderBuilder,False,False,False,True,False,public,False,Service Provider
436,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.MessageBuilder$Callback,MessageBuilder$Callback,False,True,True,False,False,public,False,Service Provider
437,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.MessageBuilder,MessageBuilder,False,False,False,True,False,public,True,Service Provider
438,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.PgpMessageBuilder,PgpMessageBuilder,False,False,False,True,False,public,False,Service Provider
440,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.SimpleMessageBuilder,SimpleMessageBuilder,False,False,False,True,False,public,False,Service Provider
442,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.TextBodyBuilder,TextBodyBuilder,False,False,False,True,False,default,False,Service Provider
503,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.mailstore.MimePartStreamParser$Par...,MimePartStreamParser$PartBuilder,True,False,True,True,False,private,False,Structurer


Load DP-CORE results on K-9 Mail case

In [7]:
class PatternInstance:
    def __init__(self, pattern: Pattern, mapping):
        self._pattern = pattern
        self._mapping = mapping
        self.members = [mapping[m.name] for m in pattern.members]
        self.name='-'.join(m.name for m in self.members)
        self.connections = [Connection(conn.name, tuple(mapping[p].name for p in conn.participants)) for conn in pattern.connections]
    def graph(self):
        dot = graphviz.Digraph(name=self.name)
        dot.attr(fontname='Arial')
        for m in self.members:
            dot.node(m.name, m.label, _attributes=m.attrs)
        for c in self.connections:
            a,b,attr = c.edge
            dot.edge(a, b, _attributes=attr)

        return dot




In [8]:

def cleanup(str):
    return ''.join(c for c in str if c.isalnum())

abstractionMapping = {
    'concrete': None,
    'abstract': 'abstract',
    'interface': 'interface'
}
for name in patternNames:
    pattern = patterns[name]
    file = open(f'k-9/patterns/detect_{name}_pattern_in_k-9-5.304_project.txt', 'r')
    lines = file.readlines()
    amount = int(lines[0].split(':')[-1])
    numMembers = len(pattern.members)
    instances = [[tuple(t.strip() for t in re.split('[\(\):]', s) if t) for s in lines[3 + (i*(2+numMembers)):3 + (i*(2+numMembers))+numMembers]] for i in range(amount)]
    instances = [i for i in instances if all(isInGroundTruth(x[2], gt_k9) for x in i)]
    print(f'{pattern.name}, amount={len(instances)}')
    lst1 = []
    lst2 = defaultdict(list)
    for instance in instances:
        memberMapping = {m[0]:Member(annot=abstractionMapping[abstraction(m[2], gt_k9)], name=cleanup(m[2]), classname=m[2], stereotype=roleStereotype(m[2], gt_k9)) for m in instance}
        # print(f'- {instance}')
        iname = '-'.join(x[2] for x in instance)
        pi = PatternInstance(pattern, memberMapping)
        roles = {cleanup(i[2]):f'{i[1]}:{str(roleStereotype(i[2], gt_k9))}' for i in instance}
        patt1 = ','.join(roles[m.name] for m in pi.members)
        lst1.append(patt1)
        for r in roles:
            p,s = tuple(roles[r].split(':'))
            lst2[p].append(s)
        # graph = pi.graph()
        # print(graph.source)
        # graph.render(outfile=f'k-9/patterns/graphs/{name}/{iname}.png')
        
    print('- pattern instances')
    ctr1 = Counter(lst1)
    for e,c in ctr1.most_common():
        print(f"  - {c}: {e}")

    print('- participants')
    for p in lst2:
        print(f"  - {p}")
        ctr2 = Counter(lst2[p])
        for e,c in ctr2.most_common():
            print(f"    - {c}: {e}")


    file.close()

Abstract Factory v1, amount=1
- pattern instances
  - 1: Concrete Factory:Coordinator,Abstract Factory:Information Holder,Product:Structurer,Abstract Product:Information Holder
- participants
  - Concrete Factory
    - 1: Coordinator
  - Abstract Factory
    - 1: Information Holder
  - Product
    - 1: Structurer
  - Abstract Product
    - 1: Information Holder
Abstract Factory v2, amount=2
- pattern instances
  - 1: Concrete Factory:Service Provider,Abstract Factory:Service Provider,Product:Information Holder,Abstract Product:Service Provider
  - 1: Concrete Factory:Structurer,Abstract Factory:Structurer,Product:Service Provider,Abstract Product:Service Provider
- participants
  - Concrete Factory
    - 1: Service Provider
    - 1: Structurer
  - Abstract Factory
    - 1: Service Provider
    - 1: Structurer
  - Product
    - 1: Information Holder
    - 1: Service Provider
  - Abstract Product
    - 2: Service Provider
Bridge v1, amount=4
- pattern instances
  - 1: Refined Abstraction

Check role stereotype inheritances

In [9]:
def graph(name, members, connections):
    dot = graphviz.Digraph(name=name)
    dot.attr(fontname='Arial')
    for m in members:
        dot.node(m.name, m.label, _attributes=m.attrs)
    for c in connections:
        a,b,attr = c.edge
        dot.edge(a, b, _attributes=attr)

    return dot

In [18]:
names = {
    'calls',
    'creates',
    'has',
    'inherits',
    'references',
    'uses'
}

for name in names:
    file = open(f'connection-descriptions/{name}.pattern', 'r')
    lines = file.readlines()
    file.close()
    pattern = Pattern(lines)

    file = open(f'k-9/connections/detect_{name}_pattern_in_k-9-5.304_project.txt', 'r')
    lines = file.readlines()
    file.close()
    amount = int(lines[0].split(':')[-1])
    numMembers = len(pattern.members)
    instances = [[tuple(t.strip() for t in re.split('[\(\):]', s) if t) for s in lines[3 + (i*(2+numMembers)):3 + (i*(2+numMembers))+numMembers]] for i in range(amount)]
    instances = [i for i in instances if all(isInGroundTruth(x[2], gt_k9) for x in i)]
    print(f'{pattern.name}, amount={len(instances)}')

    lst1 = []
    lst2 = []

    members = set()
    connections = list()

    for instance in instances:
        # print(f'- {instance}')
        memberMapping = {m[0]:Member(annot=abstraction(m[2], gt_k9), name=cleanup(m[2]), classname=m[2], stereotype=roleStereotype(m[2], gt_k9)) for m in instance}
        pi = PatternInstance(pattern, memberMapping)
        roles = {cleanup(i[2]):str(roleStereotype(i[2], gt_k9)) for i in instance}
        patt1 = ','.join(f' {c.name} '.join(roles[p] for p in c.participants) for c in pi.connections)
        lst1.append(patt1)
        # lst2.append(patt2) 
        # graph = pi.graph()
        # print(graph.source)
        # graph.render(outfile=f'k-9/patterns/graphs/{name}/{iname}.png')
        members.update(pi.members)
        connections += pi.connections
        
    ctr1 = Counter(lst1)
    # ctr2 = Counter(lst2)

    l = len("Information Holder")
    for e,c in ctr1.most_common():
        ee = e.split(f" {name} ")
        print(f"  {ee[0]:>{l}} {name} {ee[1]:<{l}}: {c}")

    # l = len("interface:Information Holder")
    # for e,c in ctr2.most_common():
    #     ee = e.split(" extends ")
    #     print(f"{ee[0]:>{l}} extends {ee[1]:<{l}}: {c}")

    prt = [c.participants for c in connections]
    a,b = tuple(zip(*prt))
    ctra = Counter(a)
    ctrb = Counter(b)
    
    print('  Participant A')
    for e,c in ctra.most_common():
        print(f"  -- {e}: {c}")
        
    print('  Participant B')
    for e,c in ctrb.most_common():
        print(f"  -- {e}: {c}")

    # g = graph(name, members, connections)
    # g.render(outfile=f'k-9/connections/graphs/{name}.png')

Inherits, amount=115
    Service Provider inherits Service Provider  : 21
          Interfacer inherits Interfacer        : 20
  Information Holder inherits Service Provider  : 15
  Information Holder inherits Information Holder: 11
         Coordinator inherits Service Provider  : 6
  Information Holder inherits Structurer        : 6
          Controller inherits Service Provider  : 5
          Structurer inherits Service Provider  : 4
          Interfacer inherits Service Provider  : 3
          Structurer inherits Information Holder: 3
    Service Provider inherits Coordinator       : 2
         Coordinator inherits Coordinator       : 2
         Coordinator inherits Information Holder: 2
         Coordinator inherits Interfacer        : 2
  Information Holder inherits Coordinator       : 2
          Interfacer inherits Coordinator       : 2
          Controller inherits Structurer        : 1
          Interfacer inherits Information Holder: 1
    Service Provider inherits Structure